In [1]:
# Import required Packages

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Declare constants
training_source = 'Udemy'
tech_name = 'FED'
no_of_courses = 14

# course id
aws_certified_cloud_practitioner = 0
aws_soultions_architect_associate = 1
aws_developer_associate = 2
aws_certified_sysops_administrator_associate = 3
aws_certified_devops_engineer_professional = 4
aws_certified_solutions_architect_professional=5

# Misc Courses 
js_complete_guide = 6
ultimate_html_developer = 7
css3_responsive_web_design = 8
sass_complete_course = 9
bootstrap_4 = 10
html_email_responsive_course =11
master_the_basics_of_html5=12
complete_nodejs_developer=13

# Initial Arrays 
exam_names = ['']*no_of_courses
exam_codes = ['']*no_of_courses
urls =['']*no_of_courses

# Course names
exam_names[aws_certified_cloud_practitioner] ='aws-certified-cloud-practitioner'
exam_names[aws_developer_associate] ='aws-developer-associate'
exam_names[aws_soultions_architect_associate] ='aws-soultions-architect-associate' 
exam_names[aws_certified_sysops_administrator_associate]='aws-certified-sysops-administrator-associate'
exam_names[aws_certified_devops_engineer_professional]='aws-certified-devops-engineer-professional'
exam_names[aws_certified_solutions_architect_professional]='aws-certified-solutions-architect-professional'

exam_names[js_complete_guide]='javascript-the-complete-guide-2020-beginner-advanced'
exam_names[ultimate_html_developer]='the-ultimate-html-developer'
exam_names[css3_responsive_web_design]='css3_responsive_web_design'
exam_names[sass_complete_course]='sass_complete_course'
exam_names[bootstrap_4]='bootstrap_4'
exam_names[html_email_responsive_course] ='HTML-EMAIL'
exam_names[master_the_basics_of_html5] ='MASTER-HTML5-CSS'
exam_names[complete_nodejs_developer] ='the-complete-nodejs-developer-course-2'

# Exam Codes
exam_codes[aws_certified_cloud_practitioner] ='CLF-C01'
exam_codes[aws_developer_associate] ='DVA-C01'
exam_codes[aws_soultions_architect_associate] ='SAA-C02'  
exam_codes[aws_certified_sysops_administrator_associate]='SOA-C01'
exam_codes[aws_certified_devops_engineer_professional]='DOP-C01'
exam_codes[aws_certified_solutions_architect_professional]='SAP-C01'

exam_codes[js_complete_guide]='JS'
exam_codes[ultimate_html_developer]='HTML'
exam_codes[css3_responsive_web_design]='CSS'
exam_codes[sass_complete_course] ='SASS'
exam_codes[bootstrap_4]='BOOTSTRAP-4'
exam_codes[html_email_responsive_course]='HTML-RESPONSIVE'
exam_codes[master_the_basics_of_html5] ='MASTER-HTML5-CSS'
exam_codes[complete_nodejs_developer] ='NodeJS'

# Course urls
urls[aws_certified_cloud_practitioner]=''

urls[aws_developer_associate] = 'https://www.udemy.com/course/aws-certified-developer-associate-dva-c01/'
urls[aws_soultions_architect_associate] ='https://www.udemy.com/course/aws-certified-solutions-architect-associate-saa-c02/'
urls[aws_certified_sysops_administrator_associate]=''

urls[aws_certified_devops_engineer_professional]=''
urls[aws_certified_solutions_architect_professional]=''
urls[js_complete_guide]='https://www.udemy.com/course/javascript-the-complete-guide-2020-beginner-advanced/'
urls[ultimate_html_developer] ='https://www.udemy.com/course/the-ultimate-html-developer/'
urls[css3_responsive_web_design] = 'https://www.udemy.com/course/css3-responsive-web-design-tutorial-with-flexbox-and-animations/'
urls[sass_complete_course]='https://www.udemy.com/course/sass-complete-course-with-project-css-preprocessor-with-projects/?src=sac&kw=sass'
urls[bootstrap_4]='https://www.udemy.com/course/bootstrap-4-from-scratch-with-5-projects/'
urls[html_email_responsive_course]='https://www.udemy.com/course/html-email-responsive-course-email-framework-email-marketing-service/'
urls[master_the_basics_of_html5]='https://www.udemy.com/course/master-the-basics-of-html5-css3-beginner-web-development/'
urls[complete_nodejs_developer]='https://www.udemy.com/course/the-complete-nodejs-developer-course-2/'


In [3]:
# Set Current Course & URL 
current_exam_scraped = complete_nodejs_developer
exam_code = exam_codes[current_exam_scraped]
exam_name = exam_names[current_exam_scraped]
start_url = urls[current_exam_scraped]

# Set the file name
file_name = training_source+'-'+exam_name

In [4]:
# Navigate to page with course contents and grab the page
with webdriver.Firefox() as driver:
    wait = WebDriverWait(driver, 10)
    driver.get(start_url)

    time.sleep(10)
 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(3)


    # And grab the page HTML source
    html_page = driver.page_source
    
    # Save a local Copy
    with open(file_name+'.html','w') as file:
        file.write(html_page)

In [5]:
# Read the Saved HTML File 
source_page =''   
# Read the local saved copy
with open(file_name+'.html','r') as file:
    source_page = file.read() 

In [6]:
# Start Processing the file
# Parse the HTML with BeautifulSoup and create a soup object
soup = BeautifulSoup(source_page)

table_columns = []
table_columns.append('TECH')
table_columns.append('EXAM CODE')
table_columns.append('TRAINING PORTAL')
table_columns.append('EXAM NAME')
table_columns.append('COURSE NAME')
table_columns.append('TITLE ORDER')
table_columns.append('TITLE')
table_columns.append('NO OF SUB-SECTIONS')
table_columns.append('SUB TITLE')
table_columns.append('DURATION')

table_data = []
title_order = 0

# Grap the course name 
course_name = soup.select('h1[class="udlite-heading-xl clp-lead__title clp-lead__title--small"]')[0].text.strip()

# Grap the course duration
sidebar = soup.select('div[class*="course-landing-page_sidebar-container"]')[0]
course_duration=sidebar.select('span[data-purpose="video-content-length"]')[0].text.replace(" on-demand video","")
course_articles = sidebar.select('span[data-purpose="num-articles"]')
if len(course_articles) == 0:
    course_subHeader = soup.select('div[class*="curriculum--sub-header"]')[0]
    subHeaders = course_subHeader.select('span[class*="curriculum--content-length--"]')[0].text
    sub_titles = subHeaders.split('•')
    course_articles = sub_titles[0]+' -- '+sub_titles[1]
else:
     course_articles=sidebar.select('span[data-purpose="num-articles"]')[0].text
        

row_data = []
# Set row data with course details
row_data.append(tech_name)
row_data.append(exam_code)
row_data.append(training_source)
row_data.append(exam_name)
row_data.append(course_name)
row_data.append(title_order)
row_data.append('')
row_data.append(course_articles)
row_data.append('')
row_data.append(course_duration)
table_data.append(row_data)
    

In [7]:
# Grab the course contents 

# Select course list
course_content = soup.select('div[class*="section--panel--"]')  
for section in course_content:
    # Set loop Variables
    row_data = []
    title_order = title_order + 1
    no_of_lectures= ''
    
    # Extract Data
    ele = section.select('h3[class*="udlite-accordion-panel-heading"]')[0]
    title = ele.select('span[class*="section--section-title--"]')[0].text
    section_content = ele.select('span[class*="section--section-content--"]')[0]
    section_content = section_content.get_text(separator=" ", strip=True).split('•',1)[0]
    section_duration = ele.select('span')[3].text
    #sub_sections = section.select('div[class*="section--lecture-title-and-description--"]')
    
    section_panels = section.select('li')
    # Set row data
    row_data.append(tech_name)
    row_data.append(exam_code)
    row_data.append(training_source)
    row_data.append(exam_name)
    row_data.append(course_name)
    row_data.append(title_order)
    row_data.append(title)
    row_data.append(no_of_lectures)
    row_data.append('')
    row_data.append(section_duration)
    table_data.append(row_data)
    
    sub_sections_count = 0
    lists_topics =section_panels;
    for section_panel in lists_topics:
        sub_row = row_data.copy()
        sub_sections_count +=1
        sub_duration = section_panel.select('span[class*="section--item-content-summary"]')[0]
        sub_title = section_panel.select('span')[0]
        print(sub_title)
        print(sub_duration)
        # set the rows values
        sub_row[8] = sub_title.text
        sub_row[9] = sub_duration.text
        table_data.append(sub_row)
    row_data[7] = sub_sections_count

<span>Welcome to the Class!</span>
<span class="section--hidden-on-mobile--171Q9 section--item-content-summary--126oS">06:58</span>
<span class="section--item-title--2k1DQ">Grab the PDF Guide</span>
<span class="section--hidden-on-mobile--171Q9 section--item-content-summary--126oS">01:04</span>
<span>Section Intro: Installing and Exploring Node.js</span>
<span class="section--hidden-on-mobile--171Q9 section--item-content-summary--126oS">00:56</span>
<span>Installing Node.js and Visual Studio Code</span>
<span class="section--hidden-on-mobile--171Q9 section--item-content-summary--126oS">08:51</span>
<span>What is Node.js?</span>
<span class="section--hidden-on-mobile--171Q9 section--item-content-summary--126oS">15:31</span>
<span>Why Should I Use Node.js?</span>
<span class="section--hidden-on-mobile--171Q9 section--item-content-summary--126oS">16:24</span>
<span>Your First Node.js Script</span>
<span class="section--hidden-on-mobile--171Q9 section--item-content-summary--126oS">06:19</s

In [8]:
# Create a Pandas DataFrame
df = pd.DataFrame(table_data, columns=table_columns)
df.to_excel(file_name+".xlsx") 
df

,TECH,EXAM CODE,TRAINING PORTAL,EXAM NAME,COURSE NAME,TITLE ORDER,TITLE,NO OF SUB-SECTIONS,SUB TITLE,DURATION
0,FED,NodeJS,Udemy,the-complete-nodejs-developer-course-2,The Complete Node.js Developer Course (3rd Edi...,0,,18 sections -- 177 lectures,,35 hours
1,FED,NodeJS,Udemy,the-complete-nodejs-developer-course-2,The Complete Node.js Developer Course (3rd Edi...,1,Welcome,2,,8min
2,FED,NodeJS,Udemy,the-complete-nodejs-developer-course-2,The Complete Node.js Developer Course (3rd Edi...,1,Welcome,,Welcome to the Class!,06:58
3,FED,NodeJS,Udemy,the-complete-nodejs-developer-course-2,The Complete Node.js Developer Course (3rd Edi...,1,Welcome,,Grab the PDF Guide,01:04
4,FED,NodeJS,Udemy,the-complete-nodejs-developer-course-2,The Complete Node.js Developer Course (3rd Edi...,2,Installing and Exploring Node.js,5,,48min
...,...,...,...,...,...,...,...,...,...,...
191,FED,NodeJS,Udemy,the-complete-nodejs-developer-course-2,The Complete Node.js Developer Course (3rd Edi...,17,Real-Time Web Applications with Socket.io (Cha...,,Deploying the Chat Application,08:06
192,FED,NodeJS,Udemy,the-complete-nodejs-developer-course-2,The Complete Node.js Developer Course (3rd Edi...,18,Wrapping Up,3,,7min
193,FED,NodeJS,Udemy,the-complete-nodejs-developer-course-2,The Complete Node.js Developer Course (3rd Edi...,18,Wrapping Up,,Section Intro,00:50
194,FED,NodeJS,Udemy,the-complete-nodejs-developer-course-2,The Complete Node.js Developer Course (3rd Edi...,18,Wrapping Up,,New Feature Ideas,03:59
